In [6]:
import numpy as np
import xarray as xr
from netCDF4 import Dataset
import pandas as pd
import matplotlib.pyplot as plt
import netCDF4 as nc
import os
import glob
import re

In [3]:
satellite_folders = ["/cluster/home/maikents/satellite_data/MODISA_L3m_CHL_R2022.0-20250313_114001", "/cluster/home/maikents/satellite_data/MODISA_L3m_CHL_R2022.0-20250313_115610", "/cluster/home/maikents/satellite_data/MODISA_L3m_CHL_R2022.0-20250313_121425"]

In [ ]:
nc_files = []
for folder in satellite_folders:
    nc_files.extend(glob.glob(os.path.join(folder, "*.nc")))

datasets = []
date_pattern = re.compile(r"(\d{4})(\d{2})(\d{2})")  #Extract YYYY MM DD

for file in nc_files:
    ds = xr.open_dataset(file)
    
    match = date_pattern.search(file)
    if match:
        year, month, day = map(int, match.groups())
        time_value = pd.Timestamp(year, month, day)
        
        ds = ds.expand_dims("time").assign_coords(time=[time_value])

        datasets.append(ds)

combined_dataset = xr.concat(datasets, dim="time")

output_file = "/cluster/home/maikents/satellite_data/combined_satellite_data.nc"
combined_dataset.to_netcdf(output_file)

print("Combined dataset saved:", output_file)
